In [0]:
#authorization from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Import Data

In [0]:
import pandas as pd
import numpy as np
import random
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV

In [0]:
data = pd.read_csv("/content/drive/My Drive/Copy of data_clean_v2.csv")

In [0]:
data = data[data.groupby(['ncodpers']).ncodpers.transform('count') == 17]
del data['Unnamed: 0']

In [0]:
#make sure certain features have correct data type
data[data.columns[19:]] = data[data.columns[19:]].astype(int)
data[data.columns[6]] = data[data.columns[6]].astype(int).astype(str)
data[data.columns[8:10]] = data[data.columns[8:10]].astype(int).astype(str)
data[data.columns[16]] = data[data.columns[16]].astype(int).astype(str)

In [0]:
#mapping categorical data to int
mapping_dict = {'ind_empleado'  : {-99:0, 'N':1, 'B':2, 'F':3, 'A':4, 'S':5},
'sexo'          : {'V':0, 'H':1 },
'ind_nuevo'     : {'0':0, '1':1},
'indrel'        : {'1':0, '99':1},
'indrel_1mes'   : {'1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'tiprel_1mes'   : { 'I':1, 'A':2, 'P':3, 'R':4, 'N':5},
'indresi'       : {'S':1,'N':2},
'indext'        : {'S':1, 'N':2},
'indfall'       : {'S':1, 'N':2},
'ind_actividad_cliente' : {'0':0, '1':1, -99:2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:3},
'pais_residencia' : {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 
                     'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 
                     'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 
                     'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 
                     'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 
                     'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 
                     'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 
                     'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 
                     'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 
                     'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27},
'canal_entrada' : {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 
                   'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 
                   'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 
                   'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 
                   'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 
                   'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 
                   'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 
                   'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 
                   'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 
                   'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 
                   'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 
                   'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 
                   'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 
                   'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 
                   'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 
                   'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11},
'nomprov' : {'ALBACETE': 76895,  'ALICANTE': 60562,  'ALMERIA': 77815,  'ASTURIAS': 83995,  'AVILA': 78525,  'BADAJOZ': 60155,  
             'BALEARS, ILLES': 114223,  'BARCELONA': 135149,  'BURGOS': 87410, 'NAVARRA' : 101850,
    'CACERES': 78691,  'CADIZ': 75397,  'CANTABRIA': 87142,  'CASTELLON': 70359,  'CEUTA': 333283, 'CIUDAD REAL': 61962,  
    'CORDOBA': 63260,  'CORUÑA, A': 103567,  'CUENCA': 70751,  'GIRONA': 100208,  'GRANADA': 80489,
    'GUADALAJARA': 100635,  'HUELVA': 75534,  'HUESCA': 80324,  'JAEN': 67016,  'LEON': 76339,  'LERIDA': 59191,  'LUGO': 68219,  
    'MADRID': 141381,  'MALAGA': 89534,  'MELILLA': 116469, 'GIPUZKOA': 101850,
    'MURCIA': 68713,  'OURENSE': 78776,  'PALENCIA': 90843,  'PALMAS, LAS': 78168,  'PONTEVEDRA': 94328,  'RIOJA, LA': 91545,  
    'SALAMANCA': 88738,  'SANTA CRUZ DE TENERIFE': 83383, 'ALAVA': 101850, 'BIZKAIA' : 101850,
    'SEGOVIA': 81287,  'SEVILLA': 94814,  'SORIA': 71615,  'TARRAGONA': 81330,  'TERUEL': 64053,  'TOLEDO': 65242,  
    'UNKNOWN': 103689,  'VALENCIA': 73463,  'VALLADOLID': 92032,  'ZAMORA': 73727,  'ZARAGOZA': 98827}
}

In [0]:
#map values
for i in range(2,19):
  if data.columns[i] in mapping_dict.keys():
    print(data.columns[i])
    data[data.columns[i]] = data[data.columns[i]].replace(mapping_dict[data.columns[i]])

ind_empleado
pais_residencia
sexo
ind_nuevo
indrel
indrel_1mes
tiprel_1mes
indresi
indext
canal_entrada
indfall
nomprov
ind_actividad_cliente
segmento


In [0]:
#sort data by ncodpers and fecha_dato
data = data.sort_values(by=['ncodpers', 'fecha_dato'])

In [0]:
#prepare features
def prepare_features(data):
  df = data[['fecha_dato', 'ncodpers','age', 'antiguedad', 'renta']]
  for col_name in ['ind_empleado', 'pais_residencia', 'sexo', 'ind_nuevo', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi',
                 'indext', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'segmento']:
    df = pd.concat([df, pd.get_dummies(data[col_name], prefix = col_name)], axis=1, sort=False)
  return df

#Prepare training and test data

In [0]:
train_index = []
last_index = []
n = data.shape[0]
for i in range(30000):
  num = random.randint(0,n)
  row = data.iloc[num,]
  if row['fecha_dato'] == "2015-01-28" or row['fecha_dato'] == '2016-05-28':
    continue
  else:
    train_index.append(num)
    last_index.append(num-1)

In [0]:
x_train = data.iloc[train_index,:]
x_test = data[data['fecha_dato'] == '2016-05-28']
x_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)

In [0]:
products_last_month_train = data.iloc[last_index,:]
products_last_month_train.reset_index(drop=True, inplace=True)
products_last_month_train = products_last_month_train.iloc[:,19:]

In [0]:
y_train = x_train.iloc[:,19:]

In [0]:
products_last_month_test = data[data['fecha_dato'] == '2016-04-28']
products_last_month_test.reset_index(drop=True, inplace=True)
products_last_month_test = products_last_month_test.iloc[:,19:]

In [0]:
y_test = x_test.iloc[:,19:]

In [0]:
#get original features
total = prepare_features(pd.concat([x_train, x_test], ignore_index=True, sort =False))

In [0]:
x_train = total[total['fecha_dato'] != "2016-05-28"]
x_test = total[total['fecha_dato'] == "2016-05-28"]

In [0]:
y_train = y_train.replace(-1,0)
y_test = y_test.replace(-1,0)

#XGBoost with Original Features Only

In [0]:
clf = OneVsRestClassifier(XGBClassifier())
clf.fit(x_train.iloc[:,2:], y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=1),
                    n_jobs=None)

In [0]:
y_pred = clf.predict(x_test.iloc[:,2:])
#precision_recall_fscore_support(y_test, y_pred,  warn_for=('precision', 'recall', 'f-score'), average = 'weighted')

In [0]:
y_pred_prob = clf.predict_proba(x_test.iloc[:,2:])

In [0]:
y_pred

array([[0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [0]:
y_pred = (y_pred_prob>0.2).astype(int)

In [0]:
y_test = y_test.to_numpy()
products_last_month_test = products_last_month_test.to_numpy()

In [0]:
y_test = y_test - products_last_month_test
y_test[y_test == -1] = 0
y_pred = y_pred - products_last_month_test
y_pred[y_pred == -1] = 0

In [0]:
total_true = y_test.sum(axis = 1).sum()
total_prediction = y_pred.sum(axis=1).sum()

In [0]:
y_pred[y_pred == 0] = -1
n_correct = np.equal(y_test,y_pred).sum(axis=1).sum()

In [0]:
#precision and recall
print(n_correct/total_prediction)
print(n_correct/total_true)

0.016289422337537336
0.5633904744674114


F1 Score

In [0]:
2*0.016289422337537336*0.5633904744674114/(0.016289422337537336+0.5633904744674114)

0.03166335569036715

#With Temporal Effects

products purchased last month (20 features)

number of products purchased last month


In [0]:
num_products_last_month_train = products_last_month_train.sum(axis=1)

In [0]:
x_train_temporal = pd.concat([x_train.iloc[:,2:], products_last_month_train.reset_index(drop = True), num_products_last_month_train.reset_index(drop = True)], axis=1) 

In [0]:
num_products_last_month_test = products_last_month_test.sum(axis=1)

In [0]:
x_test_temporal = pd.concat([x_test.iloc[:,2:].reset_index(drop = True), products_last_month_test.reset_index(drop = True),
                               num_products_last_month_test.reset_index(drop = True)], axis = 1)

In [0]:
clf = OneVsRestClassifier(XGBClassifier())
clf.fit(x_train_temporal, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=1),
                    n_jobs=None)

In [0]:
y_pred = clf.predict_proba(x_test_temporal)
#precision_recall_fscore_support(y_test, y_pred,  warn_for=('precision', 'recall', 'f-score'), average = 'weighted')

In [0]:
y_pred = (y_pred>0.2).astype(int)

In [0]:
y_pred

,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
503868,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
503869,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
503870,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
503871,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [0]:
y_pred = y_pred - products_last_month_test
y_pred[y_pred == -1] = 0

In [0]:
total_true = y_test.sum(axis = 1).sum()
total_prediction = y_pred.sum(axis=1).sum()

In [0]:
y_pred[y_pred == 0] = -1
n_correct = np.equal(y_test,y_pred).sum(axis=1).sum()

In [0]:
total_prediction

9473

In [0]:
total_true

22109

In [0]:
#precision and recall
print(n_correct/total_prediction)
print(n_correct/total_true)

0.23065554734508603
0.09882853136731648


F1 Score

In [0]:
2*0.23065554734508603*0.09882853136731648/(0.23065554734508603+0.09882853136731648)

0.13836995757076817

#Temporal Features Only

In [0]:
x_train_temporal = pd.concat([products_last_month_train, num_products_last_month_train], axis=1) 
x_test_temporal = pd.concat([ products_last_month_test.reset_index(drop = True), num_products_last_month_test.reset_index(drop=True)], axis = 1)

In [0]:
clf = OneVsRestClassifier(XGBClassifier())
clf.fit(x_train_temporal, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=1),
                    n_jobs=None)

In [0]:
y_pred = clf.predict_proba(x_test_temporal)

In [0]:
y_pred = (y_pred>0.2).astype(int)

In [0]:
y_pred = y_pred - products_last_month_test
y_pred[y_pred == -1] = 0

In [0]:
total_true = y_test.sum(axis = 1).sum()
total_prediction = y_pred.sum(axis=1).sum()

In [0]:
y_pred[y_pred == 0] = -1
n_correct = np.equal(y_test,y_pred).sum(axis=1).sum()

In [0]:
#precision and recall
print(n_correct/total_prediction)
print(n_correct/total_true)

0.22497075045963563
0.12176036908046498


F1 Score

In [0]:
2*0.22497075045963563*0.12176036908046498/(0.22497075045963563+0.12176036908046498)

0.158004402054292